<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/JaxCubicEOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/che_tools.ipynb
!pip install ipynb
!pip install dotmap

--2020-08-31 11:53:22--  https://raw.githubusercontent.com/profteachkids/CHE2064/master/che_tools.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4847 (4.7K) [text/plain]
Saving to: ‘che_tools.ipynb’

che_tools.ipynb     100%[===================>]   4.73K  --.-KB/s    in 0s      

2020-08-31 11:53:22 (56.3 MB/s) - ‘che_tools.ipynb’ saved [4847/4847]



In [2]:
import ipynb
from ipynb.fs.full.che_tools import import_ChemCad
from dotmap import DotMap
import jax.numpy as jnp
import jax
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default='plotly_dark'
R=8.314

In [3]:
props=DotMap(import_ChemCad(['Water']))

In [4]:
two_pi=2*jnp.pi
one_third = 1/3
def cubic_roots(a, b, c):
    # Returns only the real roots of cubic equations with real coefficients
    # x**3 + a x**2 + b x + c = 0

    Q = (a * a - 3 * b) / 9
    R = (2 * a * a * a - 9 * a * b + 27 * c) / 54
    det = (R * R - Q ** 3)

    if (det < 0):
      theta = jnp.arccos(R / pow(Q, 1.5))
      x=jnp.array((jnp.cos(theta/3), jnp.cos((theta+two_pi)/3), jnp.cos((theta-two_pi)/3)))
      x = -2 * jnp.sqrt(Q)*x - a/3
      return x
    else:
        A = -jnp.sign(R) * (abs(R) + jnp.sqrt(det)) ** one_third
        B = 0 if A == 0 else Q / A
        return jnp.array([(A + B) - a / 3])

In [5]:
two_pi=2*jnp.pi
one_third = 1/3
@jax.jit
def cubic_roots_jax(a, b, c):
    # Returns only the real roots of cubic equations with real coefficients
    # x**3 + a x**2 + b x + c = 0

    Q = (a * a - 3 * b) / 9
    R = (2 * a * a * a - 9 * a * b + 27 * c) / 54
    det = (R * R - Q ** 3)

    def roots3(v):
      theta = jnp.arccos(R / pow(Q, 1.5))
      x=jnp.array((jnp.cos(theta/3), jnp.cos((theta+two_pi)/3), jnp.cos((theta-two_pi)/3)))
      x = -2 * jnp.sqrt(Q)*x - a/3
      return x
    
    def roots1(v):
      A = -jnp.sign(R) * (abs(R) + jnp.sqrt(det)) ** one_third
      B = Q / A
      return jnp.array([(A + B) - a / 3, jnp.nan, jnp.nan])

    return jax.lax.cond(det < 0, roots3, roots1, (1))




In [6]:
def SRK_P(V,T,props, eos='SRK'):
  Tr = T/props.Tc
  w = props.w
  alpha = {'SRK' : (1 + (0.48 + 1.574*w - 0.176*w**2)*(1-Tr**0.5))**2}
  sigma = {'SRK' : 1}
  epsilon = {'SRK' : 0}
  omega = {'SRK' : 0.08664}
  psi = {'SRK': 0.42748}
  a = psi[eos] * alpha[eos] * R**2 * props.Tc**2 / props.Pc
  b = omega[eos] * R * props.Tc / props.Pc
  return 8.314*T/(V-b) - a/((V+epsilon[eos]*b)*(V+sigma[eos]*b))

In [7]:
b = 0.08664 * R * props.Tc / props.Pc
V=jnp.logspace(jnp.log10(1.01*b),-1, 500)


/usr/local/lib/python3.6/dist-packages/jax/lib/xla_bridge.py:130: UserWarning:

No GPU/TPU found, falling back to CPU.



In [8]:

fig=make_subplots(rows=1,cols=1)
for T in range(275,801,25):
  P=SRK_P(V,T,props)
  fig.add_scatter(x=V, y=P, mode='lines', name=f'{T}')

fig.update_layout(xaxis_title='$molar\ volume\ (m^3/mol)$',
                  yaxis_title='$Pressure\ (Pa)$')

fig.update_layout(xaxis_type='log', xaxis_range=(jnp.log10(b), -3), yaxis_range=(-50e6,200e6))
fig.show()


In [9]:
def Zv(P, T, props, eos='SRK'):
  Tr = T/props.Tc
  Pr = P/props.Pc
  w = props.w
  alpha = {'SRK' : (1 + (0.48 + 1.574*w - 0.176*w**2)*(1-Tr**0.5))**2}
  sigma = {'SRK' : 1}
  epsilon = {'SRK' : 0}
  omega = {'SRK' : 0.08664}
  psi = {'SRK': 0.42748}
  a = psi[eos] * alpha[eos] * R**2 * props.Tc**2 / props.Pc
  b = omega[eos] * R * props.Tc / props.Pc
  beta = b*P/(R*T)
  q = a/(b*R*T)

  return cubic_roots_jax(beta*(epsilon[eos]+sigma[eos])-1-beta,
                         sigma[eos]*epsilon[eos]*beta**2 - (1+beta)*beta*(epsilon[eos]+sigma[eos])+q*beta,
                         -(1+beta)*sigma[eos]*epsilon[eos]*beta**2 - q*beta**2)

In [10]:
Zv(1e5, 500, props)*R*500/1e5

DeviceArray([3.01374093e-05, 4.14211266e-02, 1.18735545e-04], dtype=float32)